# Base Load

In [1]:
import pandas as pd
import numpy as np
import os
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

import dask.dataframe as dd
from tqdm import trange
import pickle

## data 範本

In [6]:
df = pd.read_csv("./訓練資料集/split_16tag_cnt_amt.csv")

In [7]:
df.head(3)

,chid,txn_amt,txn_cnt,avr_txn_amt,2_cnt,6_cnt,10_cnt,12_cnt,13_cnt,15_cnt,...,18_amt,19_amt,21_amt,22_amt,25_amt,26_amt,36_amt,37_amt,39_amt,48_amt
0,10000000,131047.220026,28.0,4680.257858,0.0,0.0,21.0,6.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,9156.086033
1,10000001,258690.163717,66.0,3919.547935,2.0,1.0,1.0,4.0,0.0,6.0,...,0.0,0.0,19190.393873,3945.661088,0.0,25165.702415,7541.67077,34902.117580,32851.123412,52698.024693
2,10000002,9264.018807,2.0,4632.009403,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,3571.095195,0.0,0.000000,0.00000,5692.923612,0.000000,0.000000


# 將 tag 拆開成 32 維
13 - 24

1.　13.14.15　16  
2.　15.16.17　18  
3.　17.18.19　20  
4.　19.20.21　22  
5.　21.22.23　24  
6.　22.23.24　25 

## 拆分資料

In [45]:
df = dd.read_parquet("./訓練資料集/Cut_for_13dt_24dt").compute()

for i in [22]: #[13, 15, 17, 19, 21]:
    df2 = df[['chid', 'dt', 'shop_tag', 'txn_amt', 'txn_cnt']]
    df2 = df2[(df2['dt']>=i) & (df2['dt']<i+3)]
    df2 = df2.drop(columns=['dt']).groupby(by=['chid', 'shop_tag']).sum()
    df2 = df2.reset_index()
    df2.to_csv("./訓練資料集/model/0103/data/sum_at_{}_{}dt.csv".format(i, i+2), index=False)

## 轉換維度

In [46]:
p = 22
df = pd.read_csv("./訓練資料集/model/0103/data/sum_at_{}_{}dt.csv".format(p, p+2))

In [47]:
df.head(3)

,chid,shop_tag,txn_amt,txn_cnt
0,10000000,10,30947.700963,7.0
1,10000000,48,9156.086033,1.0
2,10000001,10,12847.384866,1.0


In [48]:
chid_ls = np.array(df.chid)
unique_id = df.chid.unique()
shop_tag_ls = np.array(df.shop_tag)
txn_amt_ls = np.array(df.txn_amt)
txn_cnt_ls = np.array(df.txn_cnt)
data_num = df.chid.nunique()
all_tag = np.array([2, 6, 10, 12, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48])
all_cnt_amt = [[0 for _ in range(34)] for _ in range(data_num)]

for i in trange(data_num):
    id_ = unique_id[i]
    loc = np.where(chid_ls==id_)[0]
    all_cnt_amt[i][0] = id_
    for i_loc in loc: 
        tag_loc = np.where(all_tag==shop_tag_ls[i_loc])[0][0]+1
        all_cnt_amt[i][tag_loc] = txn_cnt_ls[i_loc]
        all_cnt_amt[i][16+tag_loc] = txn_amt_ls[i_loc]
        all_cnt_amt[i][33] = txn_amt_ls[i_loc]

100%|████████████████████████████████████████████████████████████████| 468793/468793 [23:15<00:00, 335.99it/s]


In [49]:
b1 = [str(tags)+"_cnt" for tags in [2, 6, 10, 12, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48]]
b2 = [str(tags)+"_amt" for tags in [2, 6, 10, 12, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48]]
b = ["chid"]
b.extend(b1)
b.extend(b2)
b.extend(["txn_amt"])

In [50]:
a = pd.DataFrame(all_cnt_amt)
a.columns = b
a.head()

,chid,2_cnt,6_cnt,10_cnt,12_cnt,13_cnt,15_cnt,18_cnt,19_cnt,21_cnt,...,19_amt,21_amt,22_amt,25_amt,26_amt,36_amt,37_amt,39_amt,48_amt,txn_amt
0,10000000,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,9156.086033,9156.086033
1,10000001,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,10223.421432,3945.661088,0.0,0.0,0.000000,4435.600493,0.0,16653.390008,16653.390008
2,10000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,3571.095195,0.0,0.0,0.000000,0.000000,0.0,0.000000,3571.095195
3,10000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,6221.656309,10062.523765,0.0,0.000000,10062.523765
4,10000006,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,9965.795214


In [51]:
#p = 13
a.to_csv("./訓練資料集/model/0103/data/traindata_at_{}_{}dt.csv".format(p, p+2), index=False)

# 剩下的東西
dt 16

In [5]:
df = dd.read_parquet("./訓練資料集/Cut_for_13dt_24dt").compute()
df = df[df['dt']==24]
#df.to_csv("./訓練資料集/cut_by_dt-1/16dt.csv", index=False)

In [6]:
#df = pd.read_csv("./訓練資料集/cut_by_dt-1/16dt.csv")
df = df[['chid', 'txn_amt', 'shop_tag']]
df = df.groupby(by=['chid', 'shop_tag']).sum()
df = df.reset_index().sort_values(by=['chid', 'txn_amt'])

In [7]:
df.chid.nunique()

404447

In [9]:
df.head(3)

,chid,shop_tag,txn_amt
1,10000000,48,9156.086033
0,10000000,10,11660.807559
3,10000001,37,4435.600493


In [22]:
unique_id = df.chid.unique()
data_num = len(unique_id)
chid_ls = np.array(df.chid)
shop_tag_ls = np.array(df.shop_tag)
collect = [[0 for _ in range(2)] for _ in range(data_num)]

for i in trange(data_num):
    id_ = unique_id[i]
    loc = np.where(chid_ls==id_)[0]
    collect[i][0] = id_
    collect[i][1] = shop_tag_ls[loc[-1]]

100%|████████████████████████████████████████████████████████████████| 366298/366298 [08:18<00:00, 734.58it/s]


In [23]:
df2 = pd.DataFrame(collect, columns=['chid', 'top1'])
#df2.to_csv("./訓練資料集/cut_by_dt-1/16dt_top3_tag.csv", index=False)

## 找 Training data 的 y (y_tag)

In [68]:
#[15, 17, 19, 21]
p = 21
df  = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_{}_{}dt.csv".format(p, p+2))
df2 = pd.read_csv("./訓練資料集/cut_by_dt-1/{}dt_top3_tag.csv".format(p+3))

In [69]:
unique_id = df.chid.unique()
data_num = len(unique_id)
chid_ls = np.array(df2.chid)
shop_tag_ls = np.array(df2.top1)

collect = [0 for _ in range(data_num)]

for i in trange(data_num):
    id_ = unique_id[i]
    if id_ in chid_ls:
        loc = np.where(chid_ls==id_)[0]
        collect[i] = shop_tag_ls[loc][0]

100%|███████████████████████████████████████████████████████████████| 460815/460815 [04:03<00:00, 1890.82it/s]


In [71]:
df['y_tag'] = collect
df.to_csv("./訓練資料集/model/0103/data/traindata_at_{}_{}dt.csv".format(p, p+2), index=False)

# Train 處理

In [10]:
train_1 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_13_15dt.csv")
train_2 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_15_17dt.csv")
train_3 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_17_19dt.csv")
train_4 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_19_21dt.csv")
train_5 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_21_23dt.csv")

train_1 = train_1[train_1['y_tag'] > 0]
train_2 = train_2[train_2['y_tag'] > 0]
train_3 = train_3[train_3['y_tag'] > 0]
train_4 = train_4[train_4['y_tag'] > 0]
train_5 = train_5[train_5['y_tag'] > 0]

df = pd.concat([train_1, train_2, train_3, train_4, train_5]).reset_index(drop=True)
#train_1, train_2, train_3, train_4, train_5 = "", "", "", "", ""

In [13]:
df.shape

(1853573, 35)

In [14]:
df.head(3)

,chid,2_cnt,6_cnt,10_cnt,12_cnt,13_cnt,15_cnt,18_cnt,19_cnt,21_cnt,...,21_amt,22_amt,25_amt,26_amt,36_amt,37_amt,39_amt,48_amt,txn_amt,y_tag
0,10000000,0.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,18814.102542,10
1,10000001,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2136.2535,5972.879875,0.0,0.0,5972.879875,39
3,10000004,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,15271.878083,37


In [15]:
df.y_tag.nunique()

16

In [22]:
tag_unique = sorted(list(df.y_tag.unique()))
tag_unique

[2, 6, 10, 12, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48]

# Test 處理

In [48]:
test = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_22_24dt.csv")

In [3]:
test = test.groupby(by=['chid']).sum()

In [18]:
test.head(3)

,chid,2_cnt,6_cnt,10_cnt,12_cnt,13_cnt,15_cnt,18_cnt,19_cnt,21_cnt,...,19_amt,21_amt,22_amt,25_amt,26_amt,36_amt,37_amt,39_amt,48_amt,txn_amt
0,10000000,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,9156.086033,9156.086033
1,10000001,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,10223.421432,3945.661088,0.0,0.0,0.0,4435.600493,0.0,16653.390008,16653.390008
2,10000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,3571.095195,0.0,0.0,0.0,0.000000,0.0,0.000000,3571.095195


In [19]:
test.shape

(468793, 34)

In [50]:
chid_set = set(range(10000000, 10500000)) - set(test.chid)
chid_set = list(chid_set)
len(chid_set)

31207

In [51]:
new_set = []
for i in trange(len(chid_set)):
    id_ = chid_set[i]
    if id_ in list(train_5.chid):
        i_loc = np.where(np.array(train_5.chid)==id_)[0]
        test = test.append(train_5.iloc[i_loc, :-1])
        
    elif id_ in list(train_4.chid):
        i_loc = np.where(np.array(train_4.chid)==id_)[0]
        test = test.append(train_4.iloc[i_loc, :-1])
        
    elif id_ in list(train_3.chid):
        i_loc = np.where(np.array(train_3.chid)==id_)[0]
        test = test.append(train_3.iloc[i_loc, :-1])
        
    elif id_ in list(train_2.chid):
        i_loc = np.where(np.array(train_2.chid)==id_)[0]
        test = test.append(train_2.iloc[i_loc, :-1])
        
    elif id_ in list(train_1.chid):
        i_loc = np.where(np.array(train_1.chid)==id_)[0]
        test = test.append(train_1.iloc[i_loc, :-1])
        
    else:
        new_set.append(id_)

100%|█████████████████████████████████████████████████████████████████| 31207/31207 [1:21:15<00:00,  6.40it/s]


In [30]:
test.shape

(482121, 34)

In [66]:
test.to_csv("./訓練資料集/model/0103/data/traindata_at_22_24dt_filled.csv", index=False)

# 模型

## 訓練

In [16]:
import lightgbm
model = lightgbm.LGBMClassifier()

In [18]:
X = df.drop(columns=['chid', 'y_tag'])
y = df['y_tag']
model.fit(X, y)

LGBMClassifier()

In [84]:
import joblib
joblib.dump(model, './訓練資料集/model/0103/lightgbm_model.pkl')

['./訓練資料集/model/0103/lightgbm_model.pkl']

## 導入

In [52]:
import lightgbm, joblib
model = joblib.load('./訓練資料集/model/0103/lightgbm_model.pkl')

In [53]:
y_proba = model.predict_proba(test.drop(columns=['chid']))

In [54]:
y_proba.shape

(495598, 16)

In [55]:
tag_unique = [2, 6, 10, 12, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48]
result = []
for i in trange(y_proba.shape[0]):
    proba_ls = [(proba, tag) for proba, tag in zip(y_proba[i], tag_unique)]
    proba_ls = sorted(proba_ls, reverse=True)
    pred_tag_ls = [tag for proba, tag in proba_ls[:3]]
    result.append(pred_tag_ls)

100%|█████████████████████████████████████████████████████████████| 495598/495598 [00:03<00:00, 142576.52it/s]


In [56]:
result = pd.DataFrame(result, columns=['top1', 'top2', 'top3'])
result['chid'] = list(test.chid)

In [57]:
result.head(3)

,top1,top2,top3,chid
0,10,48,37,10000000
1,48,10,12,10000001
2,22,37,2,10000002


In [58]:
base = pd.read_csv("./訓練資料集/model/1231/消費總金額計算(50群, 時間加權).csv")

In [59]:
base.head(3)

,chid,top1,top2,top3
0,10128239,37,25,36
1,10077943,15,36,13
2,10277876,10,37,13


In [60]:
chid_set = set(range(10000000, 10500000)) - set(result.chid)
chid_set = list(chid_set)

In [61]:
len(chid_set)

4402

In [62]:
chid_ls = ""
data_num = len(chid_set)

for i in trange(data_num):
    base_df = base[base['chid'] == chid_set[i]]
    result = result.append(base_df)

100%|█████████████████████████████████████████████████████████████████████| 4402/4402 [00:44<00:00, 98.51it/s]


In [63]:
result.shape

(500000, 4)

In [64]:
base = base.chid
base = pd.merge(base, result, how='left', on=['chid'])

In [65]:
base = base.astype("str")
base.to_csv('./訓練資料集/model/0103/將tag攤開成32維(拯救ID篇).csv', index=False)

將攤開shop_tag的32維+總消費金額放進lightgbm，並預測 dt 22~24 的資料。  
以「消費總金額計算(50群, 時間加權).csv」為基礎，重新填補可預測資料。  
分數 : 0.6807